In [1]:
import numpy as np
import optuna.logging
import pandas as pd
import xgboost as xgb

from onnxconverter_common import FloatTensorType
from onnxmltools.convert import convert_sklearn
from sklearn.svm import SVC
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold, train_test_split

optuna.logging.set_verbosity(optuna.logging.WARNING)

C:\Users\yuki\IdeaProjects\PhishDetector\model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 特徴量データの読み込み
features_and_label: pd.DataFrame = pd.read_csv("../feature.csv").drop(columns=["domain"])

# featuresとlabelを分割
features = features_and_label.drop("label", axis=1)
labels = features_and_label[["label"]]

In [3]:
# Stratified 5-fold CV

cv_scores_columns = ["accuracy", "TPR", "TNR", "FPR", "FNR", "F1", "AUC", "TP", "TN", "FP", "FN"]
history = np.empty((0, 11))

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in kf.split(features, labels):
    x_train: pd.DataFrame = features.iloc[train_idx, :]  # 学習用特徴量
    y_train: pd.DataFrame = labels.iloc[train_idx, 0]  # 学習用ラベル
    x_test: pd.DataFrame = features.iloc[test_idx, :]  # テスト用特徴量
    y_test: pd.DataFrame = labels.iloc[test_idx, 0]  # テスト用ラベル

    dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=features.columns.tolist())
    dtest = xgb.DMatrix(x_test, label=y_test, feature_names=features.columns.tolist())

    # 訓練
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # 予測
    label_pred_prob = model.predict_proba(x_test)[:, 1]
    label_pred = (label_pred_prob > 0.5).astype(int)

    # 評価指標の計算
    tn, fp, fn, tp = confusion_matrix(y_test, label_pred).flatten()
    acc = accuracy_score(y_test, label_pred)
    tpr = tp / (tp + fn)  # recall
    tnr = tn / (tn + fp)  # specificity
    fpr = fp / (tn + fp)
    fnr = fn / (tp + fn)
    f1 = f1_score(y_test, label_pred)
    auc = roc_auc_score(y_test, label_pred_prob)

    # 評価結果の格納
    history = np.vstack([history, [acc, tpr, tnr, fpr, fnr, f1, auc, tp, tn, fp, fn]])

In [4]:
mean = np.mean(history, axis=0)
history_final = np.vstack([history, mean])

df_cv_scores = pd.DataFrame(history_final, columns=cv_scores_columns, index=["1", "2", "3", "4", "5", "平均"])
df_cv_scores

,accuracy,TPR,TNR,FPR,FNR,F1,AUC,TP,TN,FP,FN
1,0.902010,0.929648,0.874372,0.125628,0.070352,0.904645,0.956163,185.0,174.0,25.0,14.0
2,0.899497,0.944724,0.854271,0.145729,0.055276,0.903846,0.962097,188.0,170.0,29.0,11.0
3,0.884422,0.954774,0.814070,0.185930,0.045226,0.892019,0.954749,190.0,162.0,37.0,9.0
4,0.884422,0.934673,0.834171,0.165829,0.065327,0.889952,0.943234,186.0,166.0,33.0,13.0
5,0.924623,0.969849,0.879397,0.120603,0.030151,0.927885,0.969395,193.0,175.0,24.0,6.0
平均,0.898995,0.946734,0.851256,0.148744,0.053266,0.903669,0.957127,188.4,169.4,29.6,10.6


In [5]:
# 訓練
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

# 訓練
model = SVC()
model.fit(x_train, y_train.iloc[:, 0])

SVC()

In [6]:
# ONNXへの変換
initial_type = [
    ("X", FloatTensorType([None, 11])),
]
onx = convert_sklearn(model=model, initial_types=initial_type)
with open("../onnx_model/svm.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [7]:
import onnx
onnx_model = onnx.load("../onnx_model/svm.onnx")
onnx.checker.check_model(onnx_model)